In [424]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

In [446]:
address_1='/Users/krishrajpal/Documents/SummerAnalytics_Python/Train_Data.csv'
address_2='/Users/krishrajpal/Documents/SummerAnalytics_Python/Test_Data.csv'
Advertisement_Train=pd.read_csv(address_1)
Advertisement_Test=pd.read_csv(address_2)
Advertisement_Train.head(1000)

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71
...,...,...,...,...,...,...,...,...,...
995,27-09-2020,campaign 1,adgroup 3,ad 5,95,55,5.44,0,0.00
996,27-09-2020,campaign 1,adgroup 3,ad 6,190,92,7.23,6,1421.59
997,27-09-2020,campaign 1,adgroup 1,ad 7,17,8,0.20,1,96.41
998,27-09-2020,campaign 1,adgroup 2,ad 7,9,3,0.21,0,0.00


In [447]:
cols_with_missing = [col for col in Advertisement_Train.columns if Advertisement_Train[col].isnull().any()]
Advertisement_Train.drop(cols_with_missing, axis=1, inplace=True)
Advertisement_Test.drop(cols_with_missing, axis=1, inplace=True)
Advertisement_Train.drop(['date','campaign'], axis=1 ,inplace=True)
Advertisement_Test.drop(['date','campaign'], axis=1 ,inplace=True)
Advertisement_Train.head(15)

,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,adgroup 1,ad 1,24,6,0.08,0,0.00
1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,adgroup 3,ad 1,13,4,0.04,0,0.00
3,adgroup 4,ad 1,5,4,0.08,0,0.00
4,adgroup 1,ad 2,247,126,1.29,4,925.71
5,adgroup 2,ad 2,7,2,0.04,0,0.00
6,adgroup 4,ad 2,6,3,0.05,0,0.00
7,adgroup 1,ad 3,129,52,0.58,0,0.00
8,adgroup 3,ad 3,104,56,0.56,0,0.00
9,adgroup 4,ad 3,6,5,0.05,0,0.00


In [448]:
from sklearn.model_selection import train_test_split
y=Advertisement_Train.revenue
features=['adgroup','ad','impressions','clicks','cost','conversions']
X=Advertisement_Train[features]
X_train,X_valid,y_train,y_valid= train_test_split(X, y,train_size=0.8, test_size=0.2, random_state=0)
X_test = Advertisement_Test[features]

In [449]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
    RandomForestModel.fit(X_train, y_train)
    Predictions = RandomForestModel.predict(X_valid)
    return mean_absolute_error(y_valid, Predictions)

In [435]:
#drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])
print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
69.21194927522797


In [436]:
drop_X_test = X_test.select_dtypes(exclude=['object'])
RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
RandomForestModel.fit(drop_X_train, y_train)
Predictions2 = RandomForestModel.predict(drop_X_test)
revenue_prediction =pd.DataFrame(Predictions2)
revenue_prediction.columns = ["revenue"]
revenue_prediction.to_csv('RevenuePredictions2',index=False)

In [437]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['adgroup', 'ad']


In [438]:
from sklearn.preprocessing import OrdinalEncoder
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])
X_test = Advertisement_Test[features]
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
64.94144148633897


In [439]:
good_label_cols = [col for col in object_cols if 
                   set(X_test[col]).issubset(set(X_train[col]))]
bad_label_cols = list(set(object_cols)-set(good_label_cols))
print(good_label_cols)

['adgroup']


In [440]:
from sklearn.metrics import mean_squared_error
label_X_train = X_train.drop(['ad'],axis=1)
label_X_valid = X_valid.drop(['ad'],axis=1)
# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))
mse_1 = np.sqrt(mean_squared_error(y_valid,predictions_1))
print()

MAE from Approach 2 (Ordinal Encoding):
68.75101971662792



In [269]:
label_X_train = X_train.drop(['ad'],axis=1)
label_X_test = X_test.drop(['ad'],axis=1)
# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_test[good_label_cols] = ordinal_encoder.transform(X_test[good_label_cols])
RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
RandomForestModel.fit(label_X_train, y_train)
RevenuePredictions1 = RandomForestModel.predict(label_X_test)
revenue_prediction =pd.DataFrame(RevenuePredictions1)
revenue_prediction.columns = ["revenue"]
revenue_prediction.to_csv('RevenuePredictions1',index=False)

In [453]:
from sklearn.metrics import accuracy_score
RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
def score(model, title = "Default"):
    model.fit(drop_X_train, y_train)
    predictions = model.predict(drop_X_valid)
    accuracy = RandomForestModel.score(drop_X_valid,y_valid)
    print('Accuracy for', title, ':', accuracy, '\n')
score(RandomForestModel,"RandomForestClassifier")

Accuracy for RandomForestClassifier : 0.8883921627007002 



In [461]:
low_cardinality_cols = [col for col in object_cols if X[col].nunique() < 10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))
print(high_cardinality_cols,low_cardinality_cols)

['ad'] ['adgroup']


In [456]:
from sklearn.preprocessing import OneHotEncoder
OH_cols_train = X_train.drop(high_cardinality_cols,axis=1) # Your code here
OH_cols_valid = X_valid.drop(high_cardinality_cols,axis=1) # Your code here
OH_encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(OH_cols_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(OH_cols_valid[low_cardinality_cols]))
OH_cols_train.index =X_train.index
OH_cols_valid.index =X_valid.index
num_X_train = X_train.drop(object_cols,axis=1)
num_X_valid = X_valid.drop(object_cols,axis=1)
OH_X_train = pd.concat([num_X_train,OH_cols_train],axis=1)
OH_X_valid = pd.concat([num_X_valid,OH_cols_valid],axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
69.22104502681964


In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_cols_train = X.drop(high_cardinality_cols,axis=1) # Your code here
OH_cols_valid = X.drop(high_cardinality_cols,axis=1) # Your code here
OH_encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(OH_cols_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(OH_cols_valid[low_cardinality_cols]))
OH_cols_train.index = OH_X_train.index
OH_cols_valid.index = OH_X_valid.index
num_X_train = X.drop(object_cols,axis=1)
num_X_valid = X.drop(object_cols,axis=1)
OH_X_train = pd.concat([num_X_train,OH_cols_train],axis=1)
OH_X_valid = pd.concat([num_X_valid,OH_cols_valid],axis=1)

In [462]:
from sklearn.preprocessing import OneHotEncoder
OH_cols_train_1 = X.drop(high_cardinality_cols,axis=1) # Your code here
OH_cols_test_1 = X_test.drop(high_cardinality_cols,axis=1) # Your code here
OH_encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
OH_cols_train_1 = pd.DataFrame(OH_encoder.fit_transform(OH_cols_train[low_cardinality_cols]))
OH_cols_test_1 = pd.DataFrame(OH_encoder.transform(OH_cols_test[low_cardinality_cols]))
OH_cols_train_1.index = X.index
OH_cols_test_1.index = X_test.index
num_X_train_1 = X.drop(object_cols,axis=1)
num_X_test_1 = X_test.drop(object_cols,axis=1)
OH_X_train_1 = pd.concat([num_X_train,OH_cols_train],axis=1)
OH_X_test_1 = pd.concat([num_X_test,OH_cols_test],axis=1)
RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
RandomForestModel.fit(OH_X_train_1, y_train)
RevenuePredictions11 = RandomForestModel.predict(OH_X_test_1)
print(RevenuePredictions11)

KeyError: "None of [Index(['adgroup'], dtype='object')] are in the [columns]"

In [280]:
RevenuePredictions11 = RandomForestModel.predict(OH_X_test)
revenue_prediction =pd.DataFrame(RevenuePredictions3)
revenue_prediction.columns = ["revenue"]
revenue_prediction.to_csv('RevenuePredictions3',index=False)

# Cross_Validation & Pipelines {low_cardinality_colummns}

In [162]:
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == "object"]
print(numerical_cols)
print(categorical_cols)

['impressions', 'clicks', 'cost', 'conversions']
['adgroup', 'ad']


In [173]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
model = RandomForestRegressor(n_estimators=1000,random_state=0)

In [176]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_valid)
#Evaluate the MAE of Random Forest Model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 64.91426312568322


In [205]:
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])
RandomForestModel = RandomForestRegressor(n_estimators=1000, random_state=0)
RandomForestModel.fit(X_train, y_train)
X_test = Advertisement_Test[features]
RevenuePredictions1 = RandomForestModel.predict(X_test)

ValueError: could not convert string to float: 'adgroup 3'

# XGBoost Model 

In [443]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
my_model_1 = XGBRegressor(n_estimators=100,learning_rate=0.05)
my_model_1.fit(OH_X_train,y_train)
predictions_1 = my_model_1.predict(OH_X_valid)
mae_1 = mean_absolute_error(y_valid,predictions_1)
mse_1 = np.sqrt(mean_squared_error(y_valid,predictions_1))
print("Mean Absolute Error:" , mae_1)
print(mse_1)

Mean Absolute Error: 69.46027744705951
306.7832413920861


In [444]:
RevenuePredictions6 = my_model_1.predict(OH_X_test)
revenue_prediction =pd.DataFrame(RevenuePredictions6)
revenue_prediction.columns = ["revenue"]
revenue_prediction.to_csv('RevenuePredictions6',index=False)

In [422]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...


CalledProcessError: Command '['/usr/bin/java', '-version']' returned non-zero exit status 1.

In [418]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
data_train=h2o.import_file('Train_Data.cs')